Сколько страниц книги необходимо прочитать, чтобы узнать 90% всех слов?  
* https://ashirobokov.wordpress.com/category/nlp/


* Text Classification https://monkeylearn.com/text-classification/
* Классификация текстов с помощью мешка слов http://datareview.info/article/klassifikatsiya-tekstov-s-pomoshhyu-meshka-slov-rukovodstvo/
* Automated Text Classification Using Machine Learning https://towardsdatascience.com/automated-text-classification-using-machine-learning-3df4f4f9570b
* https://ru.coursera.org/lecture/vvedeniye-informatsionnyy-poisk/modiel-mieshka-slov-SgAYN
* https://www.kaggle.com/vamsi1251/bag-of-words-model/output
* https://github.com/zygmuntz/classifying-text

In [173]:
from matplotlib import pyplot
import seaborn as sns
from collections import Counter
from itertools import chain
import pandas as pd
import numpy as np

import re

import pymorphy2
from stop_words import get_stop_words

page_size = 200

In [174]:
morph = pymorphy2.MorphAnalyzer()
sw = get_stop_words('ru')
pyplot.rcParams["figure.figsize"] = (10,5)

In [175]:
def file_reader(filename, encoding='utf8', pages=0):
    output = ""
    for row in open(filename, 'r', encoding=encoding):
        output += row
        if len(output) // page_size + 1 >= pages:
            return output

In [176]:
def lemmatize(text):
    """Функция удаляет из текста стоп-слова, 
    лемматизирует текст и берёт для каждого начальную форму.
    
    # Arguments
        text: исходный текст.
    
    # Returns
        fixed_text: список лемматизированых не-стоп-слов.
    """
    letters_only = re.sub('[^a-zA-Zа-яА-Я]', ' ', text)
    fixed_text = []
    letters_only = list(filter(None, letters_only.split(' ')))
    stop_words = []
    for word in letters_only:
        # Берём первое совпадения только для простоты.
        nf = morph.parse(word)[0].normal_form
        if nf in sw:
            stop_words.append(nf)
            continue
        
        fixed_text.append(nf)
    print(f'Всего стоп-слов: {len(stop_words)}, это {len(stop_words) // page_size + 1} страниц текста')
    return fixed_text


    
def txt_to_pages(filename, encoding='utf8', pages=0):
    """Функция преобразует текст из текстового файла в "страницы"
    
    # Arguments
        filename: путь до текстового файла.
        encoding: кодировка исходного файла.
    
    # Returns
        book_pages: список списков лемматизированных слов ("страниц")    
    """
    print("Чтение файла...")
    #book_text = open(filename, mode='r', encoding=encoding).read()
    book_text = file_reader('Tolstoyi_L._Voyinaimir1._Voyina_I_Mir_Kniga_1.txt', encoding='1251', pages=pages)
    print("Лемматизация...")
    print(f'Используется слов {len(book_text)}, страниц {len(book_text) // page_size + 1}')
    book_text = lemmatize(book_text)
    
    book_pages = []
    for i in range(len(book_text) // page_size + 1):
        book_pages.append(book_text[i * page_size : i * page_size + page_size])
        if pages == len(book_pages) and pages > 0:
            break
    return book_pages

In [177]:
def calculate_hardness(coverage, percent = 0.9):
    """ Функция вычисляет "сложность" книги - 
            насколько большую часть книги необходимо прочтитать, 
            чтобы узнать percent всех используемых слов.
    
    # Arguments
        coverage: массив с информацией какой процент книги
            можем прочитать, зная слова полученные к i-ой странице.
        percent: сколько слов хотим знать.
    
    # Returns
        i: номер страницы.
        hardness: какой процент книги будет прочитан к этой странице.
    """
    for i in range(len(coverage)):
        if coverage[i] > percent:
            break
    
    hardness = (100 * i) / float(len(coverage))
    return i, hardness

In [178]:
def percentage(values, title):
    """ Функция вычисляет всё.
    
    # Arguments
        values: список "страниц" книги.
    """
    all_words = list(chain.from_iterable(values))
    result = {}
    current_text = []
    page_counter = 0
    file_name = title.lower().replace(' ', '_').replace('\'','') + '.png'
    
    counter = Counter()
    # "Счётчик" всех слов книги. 
    # Используется для подсчёта процента книги, 
    # который можно прочитать зная некоторый объём слов. 
    occurances = Counter(all_words)
    
    # i-ый элемент: какой процент книги можем прочитать зная слова полученные к i-ой странице.
    coverage = []
    
    # i-ый элемент: какой процент слов используемых в книге знаем к i-ой странице.
    uniqueness = []
    
    # i-ый элемент: сколько уникальных слов используемых в книге знаем к i-ой странице.
    total_unique_words_count = []
    
    total = float(len(all_words))
    total_unique = float(len(occurances.keys()))
    
    print('page\ttotal\tpercent of all\tpercent of book\tuniqueness')
    for page_text in values:
        # Обновим счётчик уникальных слов словами с новой страницы.
        counter.update(page_text)
        
        # Посчитаем, сколько вообще раз встречаются слова 
        # которые уже были встречены к текущей странице.
        occured_words_count = sum((occurances[w] for w in counter.keys()))

        # Какой процент от всей книги составляют слова, 
        # которые уже встретили к текущей странице.
        _coverage = occured_words_count / total
        coverage.append(_coverage)
        
        # Какой процент от всех уникальных слов встретили к текущей странице.
        _uniqueness = len(counter.keys()) / total_unique
        uniqueness.append(_uniqueness)
        
        # Какой процент от книги прошли к текущей странице.
        percent_book = page_counter / len(values)
        
        # Сколько уникальных слов будет известно к текущей странице.
        total_unique_words_count.append(len(counter.keys()))
        
        print('{0}\t{1}\t{2:2f}\t{3:2f}\t{4:2f}'.format(page_counter, 
        len(counter.keys()), 
        _coverage, 
        percent_book, 
        _uniqueness))
        
        result[page_counter] = { 
            'page': page_counter,
            'unique_words': _uniqueness, 
            'all_words': _coverage
        }
        page_counter += 1
    
    percent_df = pd.DataFrame.from_dict(result,orient='index')
    
    page, hardness = calculate_hardness(coverage)
    
    print('Всего страниц: {0}'.format(len(values)))
    print('Всего слов: {}'.format(len(all_words)))
    print('Всего уникальных слов: {}'.format(len(occurances.keys())))
    print('Прочитав {0} страниц книги ({1:.2f}% всей книги) вы узнаете {2} уникальных слов.\nК этой странице будут встречены 90% всех слов книги и {3:.2f}% уникальных.'.format(
          page,
          hardness,
          total_unique_words_count[page],
          uniqueness[page] * 100))
    
    pyplot.plot(percent_df['unique_words'], color='g', label='Уникальные слова')
    pyplot.plot(percent_df['all_words'], color='b', label='Все слова')
    pyplot.legend(loc=4)
    pyplot.title(title)
    pyplot.xlabel('Страница')
    pyplot.ylabel('Покрытие (%)')
    pyplot.show()

In [179]:
war_and_peace_pages = txt_to_pages('Tolstoyi_L._Voyinaimir1._Voyina_I_Mir_Kniga_1.txt', encoding='1251', pages=50)
#percentage(war_and_peace_pages, 'Война и мир')

Чтение файла...
Лемматизация...
Используется слов 10101, страниц 51
Всего стоп-слов: 637, это 4 страниц текста


In [86]:
a = 10
b = 3
print(f'если a/b  (с остатком)  то = {a/b}')
print(f'если a//b (без остатка) то = {a//b}')
print(f'если a%b  (остаток)     то = {a%b}')

если a/b  (с остатком)  то = 3.3333333333333335
если a//b (без остатка) то = 3
если a%b  (остаток)     то = 1
